In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql import types as t
from pyspark.sql import Window as w
from pyspark.sql import Row
from pyspark.sql.functions import udf


from pyspark.storagelevel import StorageLevel

In [0]:
spark_1 = SparkSession.builder.appName("1st app").config("spark.executor.memory","890m").config("spark.executor","2").config("spark.executor.cores","3").getOrCreate()

spark_2 = spark.newSession()
sc= spark.sparkContext

print(spark_1)
print(spark)

print(spark_2)

print(spark.sparkContext)
print(spark_2.sparkContext)


<pyspark.sql.session.SparkSession object at 0x7f817bc97c70>
<pyspark.sql.session.SparkSession object at 0x7f817bc97c70>
<pyspark.sql.session.SparkSession object at 0x7f81782eb460>
<SparkContext master=local[8] appName=Databricks Shell>
<SparkContext master=local[8] appName=Databricks Shell>

In [0]:
spark_1.sparkContext.getConf().getAll()

Out[5]: [('spark.databricks.preemption.enabled', 'true'),
 ('spark.driver.tempDirectory', '/local_disk0/tmp'),
 ('spark.sql.streaming.checkpointFileManagerClass',
 'com.databricks.spark.sql.streaming.DatabricksCheckpointFileManager'),
 ('spark.databricks.service.dbutils.repl.backend',
 'com.databricks.dbconnect.ReplDBUtils'),
 ('spark.streaming.driver.writeAheadLog.allowBatching', 'true'),
 ('spark.databricks.clusterSource', 'UI'),
 ('spark.hadoop.hive.server2.transport.mode', 'http'),
 ('spark.executor.memory', '8278m'),
 ('spark.hadoop.fs.cpfs-adl.impl.disable.cache', 'true'),
 ('spark.databricks.clusterUsageTags.hailEnabled', 'false'),
 ('spark.hadoop.fs.mcfs-s3.impl',
 'com.databricks.sql.acl.fs.ManagedCatalogFileSystem'),
 ('spark.databricks.clusterUsageTags.clusterLogDeliveryEnabled', 'false'),
 ('spark.databricks.clusterUsageTags.containerType', 'LXC'),
 ('spark.eventLog.enabled', 'false'),
 ('spark.databricks.clusterUsageTags.isIMv2Enabled', 'false'),
 ('spark.executor.tempDirectory', '/local_disk0/tmp'),
 ('spark.hadoop.mapred.output.committer.class',
 'com.databricks.backend.daemon.data.client.DirectOutputCommitter'),
 ('spark.hadoop.hive.server2.thrift.http.port', '10000'),
 ('spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version', '2'),
 ('spark.sql.allowMultipleContexts', 'false'),
 ('spark.home', '/databricks/spark'),
 ('spark.databricks.clusterUsageTags.clusterTargetWorkers', '0'),
 ('spark.hadoop.hive.server2.idle.operation.timeout', '7200000'),
 ('spark.task.reaper.enabled', 'true'),
 ('spark.storage.memoryFraction', '0.5'),
 ('spark.databricks.clusterUsageTags.clusterFirstOnDemand', '0'),
 ('spark.driver.maxResultSize', '4g'),
 ('spark.databricks.delta.multiClusterWrites.enabled', 'true'),
 ('spark.worker.cleanup.enabled', 'false'),
 ('spark.sql.legacy.createHiveTableByDefault', 'false'),
 ('spark.databricks.workspace.matplotlibInline.enabled', 'true'),
 ('spark.databricks.clusterUsageTags.enableCredentialPassthrough', 'false'),
 ('spark.databricks.clusterUsageTags.userProvidedRemoteVolumeType',
 'ebs_volume_type: GENERAL_PURPOSE_SSD\n'),
 ('spark.databricks.clusterUsageTags.enableJdbcAutoStart', 'true'),
 ('spark.hadoop.fs.wasb.impl.disable.cache', 'true'),
 ('spark.databricks.clusterUsageTags.clusterLogDestination', ''),
 ('spark.cleaner.referenceTracking.blocking', 'false'),
 ('spark.databricks.clusterUsageTags.isSingleUserCluster', 'false'),
 ('spark.databricks.clusterUsageTags.clusterState', 'Pending'),
 ('spark.databricks.tahoe.logStore.azure.class',
 'com.databricks.tahoe.store.AzureLogStore'),
 ('spark.hadoop.fs.azure.skip.metrics', 'true'),
 ('spark.hadoop.fs.s3.impl',
 'shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem'),
 ('spark.scheduler.mode', 'FAIR'),
 ('spark.sql.sources.default', 'delta'),
 ('spark.hadoop.fs.cpfs-s3n.impl',
 'com.databricks.sql.acl.fs.CredentialPassthroughFileSystem'),
 ('spark.databricks.clusterUsageTags.clusterWorkers', '0'),
 ('spark.ui.port', '47148'),
 ('spark.hadoop.fs.cpfs-adl.impl',
 'com.databricks.sql.acl.fs.CredentialPassthroughFileSystem'),
 ('spark.hadoop.fs.cpfs-abfss.impl',
 'com.databricks.sql.acl.fs.CredentialPassthroughFileSystem'),
 ('spark.databricks.clusterUsageTags.clusterId', '0717-020655-homie703'),
 ('spark.databricks.clusterUsageTags.clusterOwnerUserId', '2947462367071309'),
 ('spark.databricks.passthrough.oauth.refresher.impl',
 'com.databricks.backend.daemon.driver.credentials.OAuthTokenRefresherClient'),
 ('spark.sql.hive.metastore.sharedPrefixes',
 'org.mariadb.jdbc,com.mysql.jdbc,org.postgresql,com.microsoft.sqlserver,microsoft.sql.DateTimeOffset,microsoft.sql.Types,com.databricks,com.codahale,com.fasterxml.jackson,shaded.databricks'),
 ('spark.databricks.io.directoryCommit.enableLogicalDelete', 'false'),
 ('spark.task.reaper.killTimeout', '60s'),
 ('spark.hadoop.parquet.block.size.row.check.min', '10'),
 ('spark.hadoop.hive.server2.use.SSL', 'true'),
 ('spark.hadoop.fs.mcfs-s3a.impl',
 'com.databricks.sql.acl.fs.ManagedCatalogFile

In [0]:

%fs ls /FileStore/tables/dcad_data/


path,name,size
dbfs:/FileStore/tables/dcad_data/address.parquet,address.parquet,622007
dbfs:/FileStore/tables/dcad_data/customer.json,customer.json,40315809
dbfs:/FileStore/tables/dcad_data/item.dat,item.dat,9431085
dbfs:/FileStore/tables/dcad_data/output_dir/,output_dir/,0
dbfs:/FileStore/tables/dcad_data/web_sales.csv,web_sales.csv,146169257


In [0]:
file_location="/FileStore/tables/dcad_data/customer.json"
file_type="json"
#delimiter=","
#first_row_header="true"
#infer_schema="false"

cust_raw_df=spark.read\
.format(file_type)\
.load(file_location)

display(cust_raw_df)


address_id,birth_country,birthdate,customer_id,demographics,email_address,firstname,gender,is_preffered_customer,lastname,salutation
2133,RWANDA,1935-09-13,45721,"List(>10000, High Risk, Secondary, List(50001, 60000), 1000, 2)",Tiffany.Skinner@J.edu,Tiffany,F,N,Skinner,Miss
25398,MALAWI,1973-11-09,72411,"List(0-500, Unknown, Primary, List(170001, 180000), 3000, 4)",Dennis.Pettit@PtS7Fbdax.com,Dennis,F,Y,Pettit,Mr.
23242,JAMAICA,1979-03-30,50358,"List(0-500, Unknown, Unknown, List(120001, 130000), 2500, 3)",Issac.Saunders@Ykkz.edu,Issac,F,Y,Saunders,Mr.
37550,IRELAND,1966-10-16,96143,"List(Unknown, High Risk, College, List(20001, 30000), 1000, -1)",Carl.Mitchell@iqios5dGzoAkgSfdkO.org,Carl,F,Y,Mitchell,Dr.
22369,PAPUA NEW GUINEA,1953-05-24,93545,"List(1001-5000, Unknown, 4 yr Degree, List(180001, 190000), 8000, 1)",Antonio.Lyon@Alqrvh.org,Antonio,F,Y,Lyon,Dr.
7628,ZIMBABWE,1968-10-22,92534,"List(>10000, Unknown, Advanced Degree, List(120001, 130000), 8000, 2)",Jesse.Alexander@OU.edu,Jesse,M,N,Alexander,Dr.
24321,DJIBOUTI,1934-08-05,91798,"List(5001-10000, High Risk, Unknown, List(150001, 160000), 500, 3)",Philip.Miller@1j1uLUoeT3N.edu,Philip,M,N,Miller,Mr.
49705,LESOTHO,1958-08-02,82953,"List(501-1000, Good, 4 yr Degree, List(20001, 30000), 6000, 4)",Ronald.Harris@KUjp3Gqoqt.com,Ronald,M,N,Harris,Sir
36381,C�TE D'IVOIRE,1983-11-14,70926,"List(Unknown, Unknown, 2 yr Degree, List(110001, 120000), 2500, 0)",Myrtle.Wolford@lbKOsj.com,Myrtle,F,Y,Wolford,Dr.
40974,BANGLADESH,1976-03-27,12157,"List(>10000, Unknown, Primary, List(170001, 180000), 6000, 2)",William.Daniel@Ri4LeFs5lB4QpUfF.edu,William,F,N,Daniel,Mr.


In [0]:
.option("mode", "PERMISSIVE")
#in place of bad datatype it replaces with NULL

.option("badRecordsPath","/tmp/badRecordsPath")
# Bad records(any column having mismatched dataype ) are sent to this path.

.option("mode", "DROPMALFORMED")
#if we are not interested to capture the bad data  records only

.option("mode", "FAILFAST")
#if we dont want to load any data if data has some records mismatched

In [0]:
cust_raw_df.printSchema()



root
-- address_id: long (nullable = true)
-- birth_country: string (nullable = true)
-- birthdate: string (nullable = true)
-- customer_id: long (nullable = true)
-- demographics: struct (nullable = true)
 |-- buy_potential: string (nullable = true)
 |-- credit_rating: string (nullable = true)
 |-- education_status: string (nullable = true)
 |-- income_range: array (nullable = true)
 | |-- element: long (containsNull = true)
 |-- purchase_estimate: long (nullable = true)
 |-- vehicle_count: long (nullable = true)
-- email_address: string (nullable = true)
-- firstname: string (nullable = true)
-- gender: string (nullable = true)
-- is_preffered_customer: string (nullable = true)
-- lastname: string (nullable = true)
-- salutation: string (nullable = true)

In [0]:
cust_raw_df.schema

Out[8]: StructType(List(StructField(address_id,LongType,true),StructField(birth_country,StringType,true),StructField(birthdate,StringType,true),StructField(customer_id,LongType,true),StructField(demographics,StructType(List(StructField(buy_potential,StringType,true),StructField(credit_rating,StringType,true),StructField(education_status,StringType,true),StructField(income_range,ArrayType(LongType,true),true),StructField(purchase_estimate,LongType,true),StructField(vehicle_count,LongType,true))),true),StructField(email_address,StringType,true),StructField(firstname,StringType,true),StructField(gender,StringType,true),StructField(is_preffered_customer,StringType,true),StructField(lastname,StringType,true),StructField(salutation,StringType,true)))

In [0]:
cust_raw_df.show(7)

+----------+----------------+----------+-----------+--------------------+--------------------+---------+------+---------------------+---------+----------+
address_id| birth_country| birthdate|customer_id| demographics| email_address|firstname|gender|is_preffered_customer| lastname|salutation|
+----------+----------------+----------+-----------+--------------------+--------------------+---------+------+---------------------+---------+----------+
 2133| RWANDA|1935-09-13| 45721|{>10000, High Ris...|Tiffany.Skinner@J...| Tiffany| F| N| Skinner| Miss|
 25398| MALAWI|1973-11-09| 72411|{0-500, Unknown, ...|Dennis.Pettit@PtS...| Dennis| F| Y| Pettit| Mr.|
 23242| JAMAICA|1979-03-30| 50358|{0-500, Unknown, ...|Issac.Saunders@Yk...| Issac| F| Y| Saunders| Mr.|
 37550| IRELAND|1966-10-16| 96143|{Unknown, High Ri...|Carl.Mitchell@iqi...| Carl| F| Y| Mitchell| Dr.|
 22369|PAPUA NEW GUINEA|1953-05-24| 93545|{1001-5000, Unkno...|Antonio.Lyon@Alqr...| Antonio| F| Y| Lyon| Dr.|
 7628| ZIMBABWE|1968-10-22| 92534|{>10000, Unknown,...|Jesse.Alexander@O...| Jesse| M| N|Alexander| Dr.|
 24321| DJIBOUTI|1934-08-05| 91798|{5001-10000, High...|Philip.Miller@1j1...| Philip| M| N| Miller| Mr.|
+----------+----------------+----------+-----------+--------------------+--------------------+---------+------+---------------------+---------+----------+
only showing top 7 rows

In [0]:
#.option("inferSchema",true)


customer_DF_Schema=t.StructType([t.StructField("address_id",t.IntegerType(),True)\
                                 ,t.StructField("birth_country",t.StringType(),True)\
                                 ,t.StructField("birthdate",t.StringType(),True)\
                                 ,t.StructField("customer_id",t.IntegerType(),True)\
                                 ,t.StructField("demographics",t.StructType([t.StructField("buy_potential",t.StringType(),True)\
                                                                             ,t.StructField("credit_rating",t.StringType(),True)\
                                                                             ,t.StructField("education_status",t.StringType(),True)\
                                                                             ,t.StructField("income_range",t.ArrayType(t.LongType()),True)\
                                                                             ,t.StructField("purchase_estimate",t.LongType(),True)\
                                                                             ,t.StructField("vehicle_count",t.LongType(),True)]))\
                                 ,t.StructField("email_address",t.StringType(),True)\
                                 ,t.StructField("firstname",t.StringType(),True)\
                                 ,t.StructField("gender",t.StringType(),True)\
                                 ,t.StructField("is_preffered_customer",t.StringType(),True)\
                                 ,t.StructField("lastname",t.StringType(),True)\
                                 ,t.StructField("salutation",t.StringType(),True)\
                                ])

In [0]:
file_location="/FileStore/tables/dcad_data/customer.json"
file_type="json"
#delimiter=","
#first_row_header="true"
#infer_schema="false"

cust_raw_df_sch=spark.read\
.format(file_type)\
.schema(customer_DF_Schema)\
.load(file_location)

display(cust_raw_df_sch)

address_id,birth_country,birthdate,customer_id,demographics,email_address,firstname,gender,is_preffered_customer,lastname,salutation
2133,RWANDA,1935-09-13,45721,"List(>10000, High Risk, Secondary, List(50001, 60000), 1000, 2)",Tiffany.Skinner@J.edu,Tiffany,F,N,Skinner,Miss
25398,MALAWI,1973-11-09,72411,"List(0-500, Unknown, Primary, List(170001, 180000), 3000, 4)",Dennis.Pettit@PtS7Fbdax.com,Dennis,F,Y,Pettit,Mr.
23242,JAMAICA,1979-03-30,50358,"List(0-500, Unknown, Unknown, List(120001, 130000), 2500, 3)",Issac.Saunders@Ykkz.edu,Issac,F,Y,Saunders,Mr.
37550,IRELAND,1966-10-16,96143,"List(Unknown, High Risk, College, List(20001, 30000), 1000, -1)",Carl.Mitchell@iqios5dGzoAkgSfdkO.org,Carl,F,Y,Mitchell,Dr.
22369,PAPUA NEW GUINEA,1953-05-24,93545,"List(1001-5000, Unknown, 4 yr Degree, List(180001, 190000), 8000, 1)",Antonio.Lyon@Alqrvh.org,Antonio,F,Y,Lyon,Dr.
7628,ZIMBABWE,1968-10-22,92534,"List(>10000, Unknown, Advanced Degree, List(120001, 130000), 8000, 2)",Jesse.Alexander@OU.edu,Jesse,M,N,Alexander,Dr.
24321,DJIBOUTI,1934-08-05,91798,"List(5001-10000, High Risk, Unknown, List(150001, 160000), 500, 3)",Philip.Miller@1j1uLUoeT3N.edu,Philip,M,N,Miller,Mr.
49705,LESOTHO,1958-08-02,82953,"List(501-1000, Good, 4 yr Degree, List(20001, 30000), 6000, 4)",Ronald.Harris@KUjp3Gqoqt.com,Ronald,M,N,Harris,Sir
36381,C�TE D'IVOIRE,1983-11-14,70926,"List(Unknown, Unknown, 2 yr Degree, List(110001, 120000), 2500, 0)",Myrtle.Wolford@lbKOsj.com,Myrtle,F,Y,Wolford,Dr.
40974,BANGLADESH,1976-03-27,12157,"List(>10000, Unknown, Primary, List(170001, 180000), 6000, 2)",William.Daniel@Ri4LeFs5lB4QpUfF.edu,William,F,N,Daniel,Mr.


In [0]:
cust_raw_df_sch.printSchema()

root
-- address_id: integer (nullable = true)
-- birth_country: string (nullable = true)
-- birthdate: string (nullable = true)
-- customer_id: integer (nullable = true)
-- demographics: struct (nullable = true)
 |-- buy_potential: string (nullable = true)
 |-- credit_rating: string (nullable = true)
 |-- education_status: string (nullable = true)
 |-- income_range: array (nullable = true)
 | |-- element: long (containsNull = true)
 |-- purchase_estimate: long (nullable = true)
 |-- vehicle_count: long (nullable = true)
-- email_address: string (nullable = true)
-- firstname: string (nullable = true)
-- gender: string (nullable = true)
-- is_preffered_customer: string (nullable = true)
-- lastname: string (nullable = true)
-- salutation: string (nullable = true)

In [0]:
display(cust_raw_df_sch)

address_id,birth_country,birthdate,customer_id,demographics,email_address,firstname,gender,is_preffered_customer,lastname,salutation
2133,RWANDA,1935-09-13,45721,"List(>10000, High Risk, Secondary, List(50001, 60000), 1000, 2)",Tiffany.Skinner@J.edu,Tiffany,F,N,Skinner,Miss
25398,MALAWI,1973-11-09,72411,"List(0-500, Unknown, Primary, List(170001, 180000), 3000, 4)",Dennis.Pettit@PtS7Fbdax.com,Dennis,F,Y,Pettit,Mr.
23242,JAMAICA,1979-03-30,50358,"List(0-500, Unknown, Unknown, List(120001, 130000), 2500, 3)",Issac.Saunders@Ykkz.edu,Issac,F,Y,Saunders,Mr.
37550,IRELAND,1966-10-16,96143,"List(Unknown, High Risk, College, List(20001, 30000), 1000, -1)",Carl.Mitchell@iqios5dGzoAkgSfdkO.org,Carl,F,Y,Mitchell,Dr.
22369,PAPUA NEW GUINEA,1953-05-24,93545,"List(1001-5000, Unknown, 4 yr Degree, List(180001, 190000), 8000, 1)",Antonio.Lyon@Alqrvh.org,Antonio,F,Y,Lyon,Dr.
7628,ZIMBABWE,1968-10-22,92534,"List(>10000, Unknown, Advanced Degree, List(120001, 130000), 8000, 2)",Jesse.Alexander@OU.edu,Jesse,M,N,Alexander,Dr.
24321,DJIBOUTI,1934-08-05,91798,"List(5001-10000, High Risk, Unknown, List(150001, 160000), 500, 3)",Philip.Miller@1j1uLUoeT3N.edu,Philip,M,N,Miller,Mr.
49705,LESOTHO,1958-08-02,82953,"List(501-1000, Good, 4 yr Degree, List(20001, 30000), 6000, 4)",Ronald.Harris@KUjp3Gqoqt.com,Ronald,M,N,Harris,Sir
36381,C�TE D'IVOIRE,1983-11-14,70926,"List(Unknown, Unknown, 2 yr Degree, List(110001, 120000), 2500, 0)",Myrtle.Wolford@lbKOsj.com,Myrtle,F,Y,Wolford,Dr.
40974,BANGLADESH,1976-03-27,12157,"List(>10000, Unknown, Primary, List(170001, 180000), 6000, 2)",William.Daniel@Ri4LeFs5lB4QpUfF.edu,William,F,N,Daniel,Mr.


In [0]:
#Out[41]: StructType(List(StructField(address_id,LongType,true),StructField(birth_country,StringType,true),StructField(birthdate,StringType,true),StructField(customer_id,LongType,true),StructField(demographics,StructType(List(StructField(buy_potential,StringType,true),StructField(credit_rating,StringType,true),StructField(education_status,StringType,true),StructField(income_range,ArrayType(LongType,true),true),StructField(purchase_estimate,LongType,true),StructField(vehicle_count,LongType,true))),true),StructField(email_address,StringType,true),StructField(firstname,StringType,true),StructField(gender,StringType,true),StructField(is_preffered_customer,StringType,true),StructField(lastname,StringType,true),StructField(salutation,StringType,true)))
cust_raw_df_sch.schema

#Out[56]: StructType(List(StructField(address_id,IntegerType,true),StructField(birth_country,StringType,true),StructField(birthdate,StringType,true),StructField(customer_id,IntegerType,true),StructField(demographics,StructType(List(StructField(buy_potential,StringType,true),StructField(credit_rating,StringType,true),StructField(education_status,ArrayType(LongType,true),true),StructField(purchase_estimate,LongType,true),StructField(vehicle_count,LongType,true))),true),StructField(email_address,StringType,true),StructField(firstname,StringType,true),StructField(gender,StringType,true),StructField(is_preffered_customer,StringType,true),StructField(lastname,StringType,true),StructField(salutation,StringType,true)))

Out[14]: StructType(List(StructField(address_id,IntegerType,true),StructField(birth_country,StringType,true),StructField(birthdate,StringType,true),StructField(customer_id,IntegerType,true),StructField(demographics,StructType(List(StructField(buy_potential,StringType,true),StructField(credit_rating,StringType,true),StructField(education_status,StringType,true),StructField(income_range,ArrayType(LongType,true),true),StructField(purchase_estimate,LongType,true),StructField(vehicle_count,LongType,true))),true),StructField(email_address,StringType,true),StructField(firstname,StringType,true),StructField(gender,StringType,true),StructField(is_preffered_customer,StringType,true),StructField(lastname,StringType,true),StructField(salutation,StringType,true)))

In [0]:
from pyspark.sql import Row

columns=["first_col","second_col"]
data=[Row("arun","fname"),Row("pesari","lname")]

sc.parallelize(data).toDF().show()

+------+-----+
 _1| _2|
+------+-----+
 arun|fname|
pesari|lname|
+------+-----+

In [0]:
sc.parallelize(data).toDF(columns).show()

+---------+----------+
first_col|second_col|
+---------+----------+
 arun| fname|
 pesari| lname|
+---------+----------+

In [0]:
spark.createDataFrame(data).show()

+------+-----+
 _1| _2|
+------+-----+
 arun|fname|
pesari|lname|
+------+-----+

In [0]:
spark.createDataFrame(data,["first_col","second_column"]).show()

+---------+-------------+
first_col|second_column|
+---------+-------------+
 arun| fname|
 pesari| lname|
+---------+-------------+

In [0]:
lis=["first_col","second_column"]
spark.createDataFrame(data,lis).show()

+---------+-------------+
first_col|second_column|
+---------+-------------+
 arun| fname|
 pesari| lname|
+---------+-------------+

In [0]:
lis=["first_col","second_column"]
data=[("arun","pesar")]
spark.createDataFrame(data,lis).show()

+---------+-------------+
first_col|second_column|
+---------+-------------+
 arun| pesar|
+---------+-------------+

In [0]:
lis=["first_col"]
data=[Row("arun")]#ROW, OR TUPLE
spark.createDataFrame(data,lis).show()#IT TAKES FIRST ARGUMETN IN THE FORM OF ROW OR TUPLE OBJECTS

+---------+
first_col|
+---------+
 arun|
+---------+

In [0]:
#One Way
display(cust_raw_df_sch.select(f.col("birth_country")\
                               ,f.col("demographics.education_status")\
                               ,f.col("demographics.income_range")\
                               ,f.element_at(f.col("demographics.income_range"),1).alias("first_Value")\
                              ))

#another way
display(cust_raw_df_sch.select("birth_country"\
                               ,"demographics.education_status"\
                               ,"demographics.income_range"\
                               ,f.element_at("demographics.income_range",1).alias("first_Value")\
                              ))

#dont mix both the ways like column object and string object in select expression.

#another way
#sql expression write in f.expr("")
display(cust_raw_df_sch.select(f.expr("concat(lastname,firstname) name")))

birth_country,education_status,income_range,first_Value
RWANDA,Secondary,"List(50001, 60000)",50001
MALAWI,Primary,"List(170001, 180000)",170001
JAMAICA,Unknown,"List(120001, 130000)",120001
IRELAND,College,"List(20001, 30000)",20001
PAPUA NEW GUINEA,4 yr Degree,"List(180001, 190000)",180001
ZIMBABWE,Advanced Degree,"List(120001, 130000)",120001
DJIBOUTI,Unknown,"List(150001, 160000)",150001
LESOTHO,4 yr Degree,"List(20001, 30000)",20001
C�TE D'IVOIRE,2 yr Degree,"List(110001, 120000)",110001
BANGLADESH,Primary,"List(170001, 180000)",170001


birth_country,education_status,income_range,first_Value
RWANDA,Secondary,"List(50001, 60000)",50001
MALAWI,Primary,"List(170001, 180000)",170001
JAMAICA,Unknown,"List(120001, 130000)",120001
IRELAND,College,"List(20001, 30000)",20001
PAPUA NEW GUINEA,4 yr Degree,"List(180001, 190000)",180001
ZIMBABWE,Advanced Degree,"List(120001, 130000)",120001
DJIBOUTI,Unknown,"List(150001, 160000)",150001
LESOTHO,4 yr Degree,"List(20001, 30000)",20001
C�TE D'IVOIRE,2 yr Degree,"List(110001, 120000)",110001
BANGLADESH,Primary,"List(170001, 180000)",170001


name
SkinnerTiffany
PettitDennis
SaundersIssac
MitchellCarl
LyonAntonio
AlexanderJesse
MillerPhilip
HarrisRonald
WolfordMyrtle
DanielWilliam


In [0]:
#one of the array builtin function are used in above commnad

#element_at(arraycolumn,index)

#array_max(arraycolum)
#array_min(arraycolumn)
#array_sort(arraycolumn)


In [0]:
cust_raw_df_sch.registerTempTable("customer")

In [0]:
display(spark.sql(\
                  "select address_id\
                  ,demographics.credit_rating\
                  ,demographics.income_range\
                  ,demographics.income_range[0]\
                  ,demographics.income_range[1]\
                  from customer limit 5"\
                 )\
       )

address_id,credit_rating,income_range,demographics.income_range AS `income_range`[0],demographics.income_range AS `income_range`[1]
2133,High Risk,"List(50001, 60000)",50001,60000
25398,Unknown,"List(170001, 180000)",170001,180000
23242,Unknown,"List(120001, 130000)",120001,130000
37550,High Risk,"List(20001, 30000)",20001,30000
22369,Unknown,"List(180001, 190000)",180001,190000


In [0]:
cust_raw_df_sch.printSchema

cust_raw_df_sch.selectExpr("birthdate bday"\
                           ,"length(concat(lastname,firstname)) name_length"\
                          , "concat(lastname,firstname) name"\
                           ,"case when length(concat(firstname,lastname))>12 then 'large' else 'small' end  relative_name_length"\
                          ).show(5)

+----------+-----------+--------------+--------------------+
 bday|name_length| name|relative_name_length|
+----------+-----------+--------------+--------------------+
1935-09-13| 14|SkinnerTiffany| large|
1973-11-09| 12| PettitDennis| small|
1979-03-30| 13| SaundersIssac| large|
1966-10-16| 12| MitchellCarl| small|
1953-05-24| 11| LyonAntonio| small|
+----------+-----------+--------------+--------------------+
only showing top 5 rows

In [0]:
#takes two string arguments as input not column objects one previous and another new string
display(cust_raw_df_sch.selectExpr("birthdate bday")\
        .withColumnRenamed("bday","birthday"))

display(cust_raw_df_sch.selectExpr("birthdate bday")\
        .withColumnRenamed("djdkjdjkkdkkdk","birthday"))

birthday
1935-09-13
1973-11-09
1979-03-30
1966-10-16
1953-05-24
1968-10-22
1934-08-05
1958-08-02
1983-11-14
1976-03-27


bday
1935-09-13
1973-11-09
1979-03-30
1966-10-16
1953-05-24
1968-10-22
1934-08-05
1958-08-02
1983-11-14
1976-03-27


In [0]:
cust_raw_df_sch.columns

Out[28]: ['address_id',
 'birth_country',
 'birthdate',
 'customer_id',
 'demographics',
 'email_address',
 'firstname',
 'gender',
 'is_preffered_customer',
 'lastname',
 'salutation']

In [0]:
cust_raw_df_sch.printSchema()

root
-- address_id: integer (nullable = true)
-- birth_country: string (nullable = true)
-- birthdate: string (nullable = true)
-- customer_id: integer (nullable = true)
-- demographics: struct (nullable = true)
 |-- buy_potential: string (nullable = true)
 |-- credit_rating: string (nullable = true)
 |-- education_status: string (nullable = true)
 |-- income_range: array (nullable = true)
 | |-- element: long (containsNull = true)
 |-- purchase_estimate: long (nullable = true)
 |-- vehicle_count: long (nullable = true)
-- email_address: string (nullable = true)
-- firstname: string (nullable = true)
-- gender: string (nullable = true)
-- is_preffered_customer: string (nullable = true)
-- lastname: string (nullable = true)
-- salutation: string (nullable = true)

In [0]:
#spark-sql way of casting
cust_raw_df_sch.select(f.col("address_id").cast("string")).printSchema()

#sql way of casting best way
cust_raw_df_sch.selectExpr("cast(address_id as string)").printSchema()

root
-- address_id: string (nullable = true)

root
-- address_id: string (nullable = true)

In [0]:
len(cust_raw_df_sch.columns)

Out[31]: 11

In [0]:
cust_raw_df_sch.withColumn("newColumnName",f.lit("humane"))\
.withColumn("fullname",f.concat(f.col("firstname"),f.col("lastname")))\
.withColumn("newAdressId",f.col("address_id")+f.lit(1))\
.select(f.col("newColumnName"),f.col("fullname"),f.col("newAdressId"))\
.show(5)

#sql way of doing this
spark.sql("select 'humane' as newColumnname, concat(firstname,lastname) as fullname, address_id+2 as newAdressId from customer").show(5)

+-------------+--------------+-----------+
newColumnName| fullname|newAdressId|
+-------------+--------------+-----------+
 humane|TiffanySkinner| 2134|
 humane| DennisPettit| 25399|
 humane| IssacSaunders| 23243|
 humane| CarlMitchell| 37551|
 humane| AntonioLyon| 22370|
+-------------+--------------+-----------+
only showing top 5 rows

+-------------+--------------+-----------+
newColumnname| fullname|newAdressId|
+-------------+--------------+-----------+
 humane|TiffanySkinner| 2135|
 humane| DennisPettit| 25400|
 humane| IssacSaunders| 23244|
 humane| CarlMitchell| 37552|
 humane| AntonioLyon| 22371|
+-------------+--------------+-----------+
only showing top 5 rows

In [0]:
#%fs ls /FileStore/tables/dcad_data/web_sales.csv

In [0]:
file_location="/FileStore/tables/dcad_data/web_sales.csv"
file_format="csv"
file_delimiter=","
file_infer_schema="true"
file_header="true"

websales=spark.read\
.format(file_format)\
.option("sep",file_delimiter)\
.option("inferSchema",file_infer_schema)\
.option("header",file_header)\
.load(file_location)

#schema we are inferring though not recommended

display(websales)


ws_sold_date_sk,ws_sold_time_sk,ws_ship_date_sk,ws_item_sk,ws_bill_customer_sk,ws_bill_cdemo_sk,ws_bill_hdemo_sk,ws_bill_addr_sk,ws_ship_customer_sk,ws_ship_cdemo_sk,ws_ship_hdemo_sk,ws_ship_addr_sk,ws_web_page_sk,ws_web_site_sk,ws_ship_mode_sk,ws_warehouse_sk,ws_promo_sk,ws_order_number,ws_quantity,ws_wholesale_cost,ws_list_price,ws_sales_price,ws_ext_discount_amt,ws_ext_sales_price,ws_ext_wholesale_cost,ws_ext_list_price,ws_ext_tax,ws_coupon_amt,ws_ext_ship_cost,ws_net_paid,ws_net_paid_inc_tax,ws_net_paid_inc_ship,ws_net_paid_inc_ship_tax,ws_net_profit
2451383,73313,2451482,4591,83074,596485,1096,40907,85919,41329,1140,1351,43,4,4,5,6,1,57,33.59,59.45,38.04,1220.37,2168.28,1914.63,3388.65,50.95,1149.18,575.7,1019.1,1070.05,1594.8,1645.75,-895.53
2451383,73313,2451411,3566,83074,596485,1096,40907,85919,41329,1140,1351,28,7,3,2,271,1,38,29.83,48.92,26.41,855.38,1003.58,1133.54,1858.96,30.1,0.0,910.86,1003.58,1033.68,1914.44,1944.54,-129.96
2451383,73313,2451413,7286,83074,596485,1096,40907,85919,41329,1140,1351,58,28,10,5,300,1,32,49.72,107.89,97.1,345.28,3107.2,1591.04,3452.48,124.28,0.0,828.48,3107.2,3231.48,3935.68,4059.96,1516.16
2451383,73313,2451393,2755,83074,596485,1096,40907,85919,41329,1140,1351,2,7,6,1,63,1,65,69.18,112.07,22.41,5827.9,1456.65,4496.7,7284.55,29.13,0.0,2476.5,1456.65,1485.78,3933.15,3962.28,-3040.05
2451383,73313,2451502,2516,83074,596485,1096,40907,85919,41329,1140,1351,56,16,2,5,18,1,58,36.62,41.38,16.13,1464.5,935.54,2123.96,2400.04,84.19,0.0,167.62,935.54,1019.73,1103.16,1187.35,-1188.42
2451383,73313,2451421,16966,83074,596485,1096,40907,85919,41329,1140,1351,50,7,2,2,185,1,90,72.05,161.39,27.43,12056.4,2468.7,6484.5,14525.1,74.06,0.0,4647.6,2468.7,2542.76,7116.3,7190.36,-4015.8
2451383,73313,2451457,10402,83074,596485,1096,40907,85919,41329,1140,1351,56,16,16,3,293,1,15,83.92,174.55,69.82,1570.95,1047.3,1258.8,2618.25,20.94,0.0,392.7,1047.3,1068.24,1440.0,1460.94,-211.5
2451383,73313,2451430,1735,83074,596485,1096,40907,85919,41329,1140,1351,25,19,14,3,202,1,16,45.27,128.56,68.13,966.88,1090.08,724.32,2056.96,87.2,0.0,370.24,1090.08,1177.28,1460.32,1547.52,365.76
2451383,73313,2451458,15464,83074,596485,1096,40907,85919,41329,1140,1351,13,26,8,5,49,1,54,53.45,60.39,26.57,1826.28,1434.78,2886.3,3261.06,48.78,215.21,488.7,1219.57,1268.35,1708.27,1757.05,-1666.73
2452625,34964,2452702,8925,42296,436090,2684,37278,57428,1487225,3907,30901,9,13,16,3,267,2,47,49.64,68.99,12.41,2659.26,583.27,2333.08,3242.53,52.49,0.0,1134.58,583.27,635.76,1717.85,1770.34,-1749.81


In [0]:
websales.columns

Out[35]: ['ws_sold_date_sk',
 'ws_sold_time_sk',
 'ws_ship_date_sk',
 'ws_item_sk',
 'ws_bill_customer_sk',
 'ws_bill_cdemo_sk',
 'ws_bill_hdemo_sk',
 'ws_bill_addr_sk',
 'ws_ship_customer_sk',
 'ws_ship_cdemo_sk',
 'ws_ship_hdemo_sk',
 'ws_ship_addr_sk',
 'ws_web_page_sk',
 'ws_web_site_sk',
 'ws_ship_mode_sk',
 'ws_warehouse_sk',
 'ws_promo_sk',
 'ws_order_number',
 'ws_quantity',
 'ws_wholesale_cost',
 'ws_list_price',
 'ws_sales_price',
 'ws_ext_discount_amt',
 'ws_ext_sales_price',
 'ws_ext_wholesale_cost',
 'ws_ext_list_price',
 'ws_ext_tax',
 'ws_coupon_amt',
 'ws_ext_ship_cost',
 'ws_net_paid',
 'ws_net_paid_inc_tax',
 'ws_net_paid_inc_ship',
 'ws_net_paid_inc_ship_tax',
 'ws_net_profit']

In [0]:
display(\
        websales.select("ws_sold_date_sk", 'ws_sold_time_sk',)\
        .withColumn("newColumn",f.col("ws_sold_date_sk")+f.col( 'ws_sold_time_sk')+f.lit(1))\
        .withColumn("lengthcolum",f.expr("length(newColumn)"))\
       )


ws_sold_date_sk,ws_sold_time_sk,newColumn,lengthcolum
2451383,73313,2524697,7
2451383,73313,2524697,7
2451383,73313,2524697,7
2451383,73313,2524697,7
2451383,73313,2524697,7
2451383,73313,2524697,7
2451383,73313,2524697,7
2451383,73313,2524697,7
2451383,73313,2524697,7
2452625,34964,2487590,7


In [0]:
# LINEAGE
# =======
# set of steps(TRANSFORMATIONS) required to generate an RDD
# rdd.toDebugSchema()
# from bottom to top(i.e from getting data from a file to creating a required RDD)
# Bascially lineage is a logical plan

# DAG
# ===
# It is a physical plan of stages and stage boundaries.
# when action is hit ,a logical plan(lineage) gets submitted to catalyst optimzer and optimized logical plan is given to DAG Scheduler when then generated the Directed Acyclic Graph for the Job, which includes Stages, Boundaries.
# DAG Decides which stages can be run in parallel, which are not.
# DAG is more comprehensive



websales.explain('formatted')

== Physical Plan ==
Scan csv (1)


(1) Scan csv 
Output [34]: [ws_sold_date_sk#611, ws_sold_time_sk#612, ws_ship_date_sk#613, ws_item_sk#614, ws_bill_customer_sk#615, ws_bill_cdemo_sk#616, ws_bill_hdemo_sk#617, ws_bill_addr_sk#618, ws_ship_customer_sk#619, ws_ship_cdemo_sk#620, ws_ship_hdemo_sk#621, ws_ship_addr_sk#622, ws_web_page_sk#623, ws_web_site_sk#624, ws_ship_mode_sk#625, ws_warehouse_sk#626, ws_promo_sk#627, ws_order_number#628, ws_quantity#629, ws_wholesale_cost#630, ws_list_price#631, ws_sales_price#632, ws_ext_discount_amt#633, ws_ext_sales_price#634, ws_ext_wholesale_cost#635, ws_ext_list_price#636, ws_ext_tax#637, ws_coupon_amt#638, ws_ext_ship_cost#639, ws_net_paid#640, ws_net_paid_inc_tax#641, ws_net_paid_inc_ship#642, ws_net_paid_inc_ship_tax#643, ws_net_profit#644]
Batched: false
Location: InMemoryFileIndex [dbfs:/FileStore/tables/dcad_data/web_sales.csv]
ReadSchema: struct<ws_sold_date_sk:int,ws_sold_time_sk:int,ws_ship_date_sk:int,ws_item_sk:int,ws_bill_customer_sk:int,ws_bill_cdemo_sk:int,ws_bill_hdemo_sk:int,ws_bill_addr_sk:int,ws_ship_customer_sk:int,ws_ship_cdemo_sk:int,ws_ship_hdemo_sk:int,ws_ship_addr_sk:int,ws_web_page_sk:int,ws_web_site_sk:int,ws_ship_mode_sk:int,ws_warehouse_sk:int,ws_promo_sk:int,ws_order_number:int,ws_quantity:int,ws_wholesale_cost:double,ws_list_price:double,ws_sales_price:double,ws_ext_discount_amt:double,ws_ext_sales_price:double,ws_ext_wholesale_cost:double,ws_ext_list_price:double,ws_ext_tax:double,ws_coupon_amt:double,ws_ext_ship_cost:double,ws_net_paid:double,ws_net_paid_inc_tax:double,ws_net_paid_inc_ship:double,ws_net_paid_inc_ship_tax:double,ws_net_profit:double>

In [0]:
display(\
        cust_raw_df_sch.filter(f.col("gender") == f.lit("M"))\
        .filter(f.col("birth_country") == "UGANDA")\
        .filter(f.month(f.col("birthdate"))==1)\
       )
  #here month takes column object not string      

address_id,birth_country,birthdate,customer_id,demographics,email_address,firstname,gender,is_preffered_customer,lastname,salutation
31740,UGANDA,1942-01-24,38003,"List(5001-10000, High Risk, Unknown, List(170001, 180000), 500, 3)",William.Freeman@8.com,William,M,N,Freeman,Sir
11540,UGANDA,1924-01-15,51202,"List(>10000, Good, 4 yr Degree, List(40001, 50000), 5000, 0)",Frederick.Calderon@PD.com,Frederick,M,N,Calderon,Sir
27145,UGANDA,1943-01-27,53547,"List(1001-5000, Low Risk, 4 yr Degree, List(90001, 100000), 5000, 3)",Levi.Schaefer@Jl5BGfgynd.com,Levi,M,N,Schaefer,Sir
45771,UGANDA,1937-01-03,63312,"List(501-1000, Low Risk, Advanced Degree, List(180001, 190000), 2000, 1)",Elisa.Booker@7.edu,Elisa,M,Y,Booker,Ms.
41606,UGANDA,1971-01-30,86811,"List(Unknown, Unknown, 2 yr Degree, List(120001, 130000), 7000, 1)",Sharon.Ward@Q2XUoCuenF2O.edu,Sharon,M,N,Ward,Mrs.
36139,UGANDA,1988-01-13,94814,"List(5001-10000, High Risk, Advanced Degree, List(140001, 150000), 7000, 1)",Robert.Wright@xDBKFOT9.edu,Robert,M,Y,Wright,Dr.
45779,UGANDA,1951-01-29,37665,"List(0-500, Good, Advanced Degree, List(180001, 190000), 2500, -1)",Russell.Jackson@xor1zIt.com,Russell,M,N,Jackson,Sir
35440,UGANDA,1964-01-05,88678,"List(501-1000, Low Risk, Advanced Degree, List(60001, 70000), 1000, 4)",Nathaniel.Calderon@SxRc9svm6BSfsC.org,Nathaniel,M,N,Calderon,Mr.
42423,UGANDA,1963-01-06,60387,"List(0-500, Low Risk, 2 yr Degree, List(40001, 50000), 9000, 0)",Rozanne.Patterson@Hi1KHKYioN.edu,Rozanne,M,N,Patterson,Dr.
22857,UGANDA,1991-01-09,4559,"List(501-1000, High Risk, College, List(190001, 200000), 8500, 3)",Luis.Salazar@zS7Q.org,Luis,M,N,Salazar,Sir


In [0]:
# Narrow Transfromation or Narrow Dependencies
# Where one partition of a dataframe produces only one dataframe 

In [0]:
cust_raw_df_sch.select("birth_country").distinct().show()

In [0]:
cust_raw_df_sch.dropDuplicates(["birth_country"]).show()
#first one is taken all the duplicate column named rows 

In [0]:
# df.dropDuplicates == df.distinct()
# both are equivlalent

In [0]:
display(\
        cust_raw_df_sch.selectExpr("salutation","firstname","lastname","email_address","year(birthdate) birthyear ")\
       )

In [0]:
dfn=cust_raw_df_sch.selectExpr("salutation","firstname","lastname","email_address","year(birthdate) birthyear ")

#isNull
display(\
        dfn.filter(f.col("salutation").isNull())\
       )

#isNotNull
display(\
        dfn.filter(f.col("salutation").isNotNull())\
       )

salutation,firstname,lastname,email_address,birthyear
null,Frederick,null,null,null
null,Henry,null,null,null
null,Linda,Wells,Linda.Wells@EalZgHD7DGZqk.org,null
null,null,Owens,Rita.Owens@u4BFK.edu,null
null,Michelle,Gist,null,null
null,Christine,Jimenez,Christine.Jimenez@fnVjrDi4l9xSZX4.com,null
null,Ruth,null,null,null
null,Adrian,null,null,null
null,James,null,James.Burnette@VanLv.edu,null
null,null,null,null,null


salutation,firstname,lastname,email_address,birthyear
Miss,Tiffany,Skinner,Tiffany.Skinner@J.edu,1935
Mr.,Dennis,Pettit,Dennis.Pettit@PtS7Fbdax.com,1973
Mr.,Issac,Saunders,Issac.Saunders@Ykkz.edu,1979
Dr.,Carl,Mitchell,Carl.Mitchell@iqios5dGzoAkgSfdkO.org,1966
Dr.,Antonio,Lyon,Antonio.Lyon@Alqrvh.org,1953
Dr.,Jesse,Alexander,Jesse.Alexander@OU.edu,1968
Mr.,Philip,Miller,Philip.Miller@1j1uLUoeT3N.edu,1934
Sir,Ronald,Harris,Ronald.Harris@KUjp3Gqoqt.com,1958
Dr.,Myrtle,Wolford,Myrtle.Wolford@lbKOsj.com,1983
Mr.,William,Daniel,William.Daniel@Ri4LeFs5lB4QpUfF.edu,1976


In [0]:
display(\
        dfn.na.drop("all")
       )

In [0]:
display(\
        dfn.na.drop("any")
       )

In [0]:
display(\
        dfn.na.drop(how="all",subset=["salutation","birthyear"])\
       )

In [0]:
display(\
        dfn.na.drop(how="any",subset=["salutation","birthyear"])\
       )

In [0]:
display(\
        dfn.na.fill("NULLED FILLED")\
       )

In [0]:
display(\
        dfn.na.fill(20)\
       )

In [0]:
display(\
        dfn.na.fill({"salutation":"AAAAAA","birthyear":9999})\
       )

In [0]:
display(\
        dfn.na.drop(how="any").sort(f.col("birthyear").asc(),f.col("firstname").desc())\
       )

In [0]:
websales.printSchema()

In [0]:
custpurchases=websales.selectExpr("ws_bill_customer_sk customer_id", "ws_item_sk item_id")
display(custpurchases)

In [0]:
#shuffle takes place leading to stage boundary
#child partitions created from multiple parent partitions, this transformations are called Wide Transformation or Wide Dependency
#Wide transformations are network intensive

#groupBy
#Join
#Distinct
#Repartition
#Colesce
#sort

display(\
        custpurchases.groupBy(f.col("customer_id")).agg(f.count(f.col("item_id")).alias("counter"),f.max(f.col("item_id")).alias("maximum"))\
       )

In [0]:
display(\
        custpurchases.agg(f.count(f.col("item_id")).alias("counter")\
                          ,f.max(f.col("item_id")).alias("maximum")\
                          ,f.min(f.col("item_id")).alias("minimum")\
                          ,f.avg(f.col("item_id")).alias("average")\
                          ,f.mean(f.col("item_id")).alias("mean")\
                         )
       )

In [0]:
display(\
        custpurchases.groupBy(f.col("customer_id")).agg(f.count(f.col("item_id")).alias("counter")\
                          ,f.max(f.col("item_id")).alias("maximum")\
                          ,f.min(f.col("item_id")).alias("minimum")\
                          ,f.avg(f.col("item_id")).alias("average")\
                          ,f.mean(f.col("item_id")).alias("mean")\
                         )
       )

In [0]:
display(\
        dfn\
       )
        

In [0]:
display(\
        dfn.select(f.count("lastname"))\
       )

display(\
        dfn.select(f.count("*"))\
       )
dfn.select("lastname").filter(f.col("lastname").isNull()).na.fill({"lastname":"xxxx"}).select(f.count("*")).show()
     
#sum above and below to get 100000

In [0]:
display(\
        dfn.describe()\
       )

In [0]:
display(\
        dfn.describe("birthyear")\
       )

In [0]:
display(\
        dfn.summary("mean")\
       )

In [0]:
%fs ls /FileStore/tables/dcad_data/

In [0]:
file_location="/FileStore/tables/dcad_data/address.parquet"
file_format="parquet"
#file_delimiter=","
#file_infer_schema="true"
#file_header="true"

address=spark.read\
.format(file_format)\
.load(file_location)

#schema we are inferring though not recommended

display(address)
display(cust_raw_df_sch)

In [0]:
display(\
        cust_raw_df_sch.join(address,cust_raw_df_sch.address_id==address.address_id,"outer")\
       )

#inner,left,outer,right,left-semi,left-anti

#left-semi
a
10
10
12
12
13
14

b
10
10
12
14

c(left-semi)
10
10
12
12
14

d(left-anti)
13


In [0]:
# combining two dataframes literally
# df1.union(df2)

In [0]:
# dataframe.persist()
# dataframe.cache()
# The persisitance level for both the commands is MEMORY_AND_DISK

# for persist
# we could give additional persistence levels as
#      MEMORY_ONLY, MEMORY_ONLY_2
#      MEMORY_ONLY_SER, MEMORY_ONLY_SER_2
#      MEMORY_AND_DISK, MEMORY_AND_DISK_2
#      MEMORY_AND_DISK_SER, MEMORY_AND_DISK_SER_2
#      DISK_ONLY, DISK_ONLY_2
#      DISK_ONLY_SER, DISK_ONLY_SER_2
#      


# FOR RDD
# rdd.persist()
# rdd.cache()
# DEFAULT persistance level is "MEMORY_ONLY"

#spark automatically check cache usage on each nodes and drops old data partitionsin Least Recently Used  Fashion

#to manually remove an RDD or Dataframe
#to Unpersist
# dataframe.unpersist


# the module is present in pyspark.STO

In [0]:
from pyspark.storagelevel import StorageLevel
dfn.persist(StorageLevel.MEMORY_AND_DISK)

Out[7]: DataFrame[salutation: string, firstname: string, lastname: string, email_address: string, birthyear: int]

In [0]:
dfn.storageLevel
#class pyspark.StorageLevel(useDisk, useMemory, useOffHeap, deserialized, replication = 1)

Out[11]: StorageLevel(True, True, False, False, 1)

In [0]:
dfn.cache()

In [0]:
dfn.count()

In [0]:
dfn.count()

In [0]:
dfn.count()

In [0]:
dir(spark)

In [0]:
file_location="/FileStore/tables/person-1"
file_type="csv"
delimiter=","
first_row_header="true"
schema=t.StructType([t.StructField("id",t.IntegerType(), False)\
                         ,t.StructField("fname",t.StringType(), True)\
                         ,t.StructField("lname",t.StringType(), True)\
                         ,t.StructField("deptid",t.IntegerType(), False)\
                         ,t.StructField("salary",t.IntegerType(), True)])

w=spark.read\
.format(file_type)\
.option("sep",delimiter)\
.option("header",first_row_header)\
.schema(schema)\
.load(file_location)

w.show(1)
w.printSchema()
lis=[t.Row(1,"bolla","rash",4,1000),t.Row(1,"bolla","shafg",4,4000),t.Row(2,"sdfd","fddfd",4,2000)]
x=spark.createDataFrame(lis,schema)

z=w.union(x)
z.registerTempTable("z")

display(z)


In [0]:
spark.sql("SELECT *\
,ROW_NUMBER() OVER (ORDER BY salary DESC) AS SalaryOrder \
,ROW_NUMBER() OVER (PARTITION BY deptid ORDER BY salary DESC) AS dept_wise_salary \
from z \
ORDER BY deptid").show()

In [0]:
  z.withColumn("Highest Salary" , f.row_number().over(w.orderBy(f.col("salary"))))\
  .withColumn("Deptwise Salary", f.row_number().over(w.partitionBy(f.col("deptid")).orderBy(f.col("salary").desc())))\
  .sort(f.col("deptid"))\
  .show()

In [0]:
spark.sql("SELECT *\
,RANK() OVER (ORDER BY salary DESC) AS SalaryOrder \
,RANK() OVER (PARTITION BY deptid ORDER BY salary DESC) AS dept_wise_salary \
from z \
ORDER BY deptid").show()

In [0]:
  z.withColumn("Highest Salary" , f.rank().over(w.orderBy(f.col("salary"))))\
  .withColumn("Deptwise Salary", f.rank().over(w.partitionBy(f.col("deptid")).orderBy(f.col("salary").desc())))\
  .sort(f.col("deptid"))\
  .show()

In [0]:
  z.withColumn("Highest Salary" , f.dense_rank().over(w.orderBy(f.col("salary"))))\
  .withColumn("Deptwise Salary", f.dense_rank().over(w.partitionBy(f.col("deptid")).orderBy(f.col("salary").desc())))\
  .sort(f.col("deptid"))\
  .show()

In [0]:
  z.withColumn("previous Salary" , f.lag(f.col("salary")).over(w.partitionBy("deptid").orderBy(f.col("salary").desc())))\
  .withColumn("next Salary", f.lead(f.col("salary")).over(w.partitionBy(f.col("deptid")).orderBy(f.col("salary").desc())))\
  .withColumn("previous Salary_2" , f.lag(f.col("salary"),2).over(w.partitionBy("deptid").orderBy(f.col("salary").desc())))\
  .withColumn("next Salary_2", f.lead(f.col("salary"),2).over(w.partitionBy(f.col("deptid")).orderBy(f.col("salary").desc())))\
   .withColumn("previous Salary_3" , f.lag(f.col("salary"),3).over(w.partitionBy("deptid").orderBy(f.col("salary").desc())))\
  .withColumn("next Salary_3", f.lead(f.col("salary"),3).over(w.partitionBy(f.col("deptid")).orderBy(f.col("salary").desc())))\
  .sort(f.col("deptid"),f.col("salary").desc())\
  .show()

In [0]:
dir(w)

In [0]:
  z.withColumn("first Salary" , f.first(f.col("salary")).over(w.partitionBy(f.col("deptid")).orderBy(f.col("salary"))))\
  .withColumn("last Salary", f.last(f.col("salary")).over(w.partitionBy(f.col("deptid")).orderBy(f.col("salary").desc())))\
  .sort(f.col("deptid"))\
  .show()

In [0]:
z\
.withColumn("Dept-wise-with_Range_frame", f.sum(f.col("salary")).over(w.partitionBy(f.col("deptid")).orderBy(f.col("id").desc()).rangeBetween(w.unboundedPreceding, w.unboundedFollowing)))\
.withColumn("Dept-wise_with_Rows_frame", f.sum(f.col("salary")).over(w.partitionBy(f.col("deptid")).orderBy(f.col("id").desc()).rowsBetween(w.unboundedPreceding, w.unboundedFollowing)))\
.sort(f.col("deptid"))\
.show()

In [0]:

z\
.repartition(4)\
.write\
.partitionBy("deptid")\
.mode("overwrite")\
.format("parquet")\
.option("compression","snappy")\
.option("path","/FileStore/tables/dcad_data/output_dir")\
.save()


# Default file formats is Parquet
# Default file compression is Snappy

# Repartition vs Colesce
# ======================
# increase and decrese the number of partitions in repatitions
# Doesn't avoid shuffling
# No Data Skewing 
# will have network latency issue

# Coalesce
# ========
# only decrease the number of partitions 
# avoids shuffling
# Data skewing in partitions
# will not have network latency issues




In [0]:
%fs ls /FileStore/tables/dcad_data/output_dir

In [0]:
cust_raw_df_sch.rdd.getNumPartitions()

In [0]:
def concatee(x,y):
  return [x,y]


#registering to use in spark.sql IN SQL EXPRESSION
spark.udf.register("sql_udffunc",concatee)
cust_raw_df_sch.selectExpr("firstname","lastname",sql_udffunc("firstname","lastname")).show(3)

##registering to use in spark.sql IN DATAFRAMES
sql_udffunc=udf(concatee,t.ArrayType(t.StringType()))
cust_raw_df_sch.select("firstname","lastname",udffunc("firstname","lastname")).show(3)

In [0]:
Df.write.format('jdbc').options(
      url='jdbc:oracle:thin:@192.168.11.100:1521:ORCL',
      driver='oracle.jdbc.driver.OracleDriver',
      dbtable='testschema.test',
      user='testschema',
      password='password').mode('overwrite').save()



# SaveModes
# =========
# SaveMode.Overwrite
# SaveMode.Append
# SaveMode.Ignore
# SaveMode.ErrorIfExists

In [0]:
#one way
id=3
query = "SELECT * FROM table1 where param={}".format(id) 
sqlContext.sql(query)

#another way
sqlContext.sql(s"SELECT count from mytable WHERE id=$id")

In [0]:
# collect
#========
# When a file sitting inside a cluster, when calling a collect operation on dataframe......as file might be bigger than driver memory...Hence, driver OOM

# broadcast join
#================
# when one file is smaller and other one is larger, when we perform broadcast join, the smaller file (which when merged upon coming to driver) that merged dataframe size is greater than driver memory, it causes Driver OOM

#how to avoid this
#sql.auto broadcastjointhreshold should be small...so that no file above this parameter size can go to driver 

In [0]:
RESEVERD MEMORY
#==============
In executor, RESEVERD MEMORY= 300MB (What ever the size of executor is)
#it is better not tamper with this memory. #It is hardcoded in spark source code


USER MEMORY
#==========
#UDF, some user defined datastructures are stored here
  25% of (executor-reseverd) is stored here

  #at the run -time spark will not extend this limit. If it extends you will get Executor OOM
  
  
SPARK MEMORY FRACTION
=====================
  75% of (executor-reseverd) is stored here
  Storage memory + execution memory
  
  storage memory:: CACHE objects(RDD / Dataframes) are stored
                   Broadcasted RDD / Dataframes are stored here
                   Has the option of spilling into disk
                  storage memory blocks CANNOT evict already existing execution memory blocks
  
  execution memory:: a lot of temporary data gets generated is stored here.
                  Has the option of spilling into disk'
                  execution memory blocks CAN evict already existing storage memory blocks
          
  the boundary between storage and execution memory is pretty movable and configured.
  

In [0]:
resource we have
6 machines
16 cores(96 cores)
64 gb ram/machine

our options
===========
smallest size
biggest size
right approac

smallest size
=============
executor takes 1 core
every node has 16 cores so 16 executors==>96 execuotors
every executor has 4gb ram

no threading/parallelization capability here
so no good strategy

biggest size
============
executor takes 16 cores
so every node has 1 execuor==> 6 executors
executor has 64 gb ram

all trying to read parttion from hdfs ==> high hdfs io contention==>reads and writes becomes low
no resources for os resources
memory overhead is huge

right approach
==============
cores side:
    1 cores on each machine to OS purposes

    remaining 90 cores wholly
    every machines has 15 cores

    executor takes 5 cores per machine(standard)

    3 execuotors per machine

memory side:
    1gb per machine for OS and other purposes
    63 gb ram available now
    memory required per executor = 21 gb(63/3)-2gb(leaving some ram to spark and yarn overhead)
    19 gb per executor


In [0]:
# if we want the information to be available in all available workers, so that execuor can use and process
# that is done by broadcast variables

# broadcast variables are readonly variables

# broadcastvariable=sc.broadcast(m)==> reference like bc.value()
# if we need workers stop using the broadcast variable, we could call broadcastvariable.destroy() method

#by default broadcasted variables are cached we could use unpersist() to remove them

In [0]:
# variables which are used to aggregate information for executors in our driver
#
# these many records are corrupt, these are used for status of jobs
# while processing records some rows may be processing fine some may be corrupted, how many records were corrupt wholly
#
# execuors are not meant to read
# driver is the one which is supposed to read at the end of job


# accuumlaor_variable=sc.accumulator()
# accumulator_variable.value

In [0]:
from pyspark.sql.functions import when
df2 = df.withColumn("new_gender", when(df.gender == "M","Male")
                                 .when(df.gender == "F","Female")
                                 .when(df.gender.isNull() ,"")
                                 .otherwise(df.gender))

In [0]:
print()

In [0]:
%fs

ls

path,name,size
dbfs:/FileStore/,FileStore/,0
dbfs:/databricks-datasets/,databricks-datasets/,0
dbfs:/databricks-results/,databricks-results/,0
